# Linear Algebra and Optimisation

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Week 1

In this lab we will practice minimising a cost function with gradient descent.

### Assumed knowledge
- Linear algebra (see Sam Roweis' notes, linked below, for matrix calculus tips)
- Python programming
- Preferably: Using numpy for matrix calculations (precourse material)

### After this lab, you should be comfortable with:
- Using numpy ndarrays for matrix calculations
- Using scipy.optimise routines to minimise a cost function, with and without a gradient
- Randomly generating input values for testing

## Pre-lab notes
In this lab, you will apply linear algebra to to minimise a cost function in three steps: implementing the cost function, implementing a gradient function, and applying gradient descent. We will be doing this to solve problems throughout the course.

As in all labs, feel free to skip questions if you get stuck, and ask your tutor if you have any questions!

A note on style: in this course we emphasise *functional decomposition* in code style. Avoid using global variables, and remember that often splitting code off into separate functions can make it more readable and testable. (Jupyter notebooks let you call functions defined in previous cells.)

$\newcommand{\trace}[1]{\operatorname{tr}\left\{#1\right\}}$
$\newcommand{\Norm}[1]{\lVert#1\rVert}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\inner}[2]{\langle #1, #2 \rangle}$
$\newcommand{\DD}{\mathscr{D}}$
$\newcommand{\grad}[1]{\operatorname{grad}#1}$
$\DeclareMathOperator*{\argmin}{arg\,min}$

Setting up python environment (this cell contains Latex macros).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.optimize as opt
import time

%matplotlib inline

A *cost function* or *loss function* is the function we want to minimize in a given problem. For example, it might measure the error between the indicator values our models predicts and the true values for the training data. In this lab, we consider a toy example. We will define a cost function $f(X)$ where $X$ is a $n\times p$ matrix.

If $A$ is a square matrix, then we write $\trace{A}$ for its trace. Let $ \Norm{A}_F = \sqrt{\trace{A^T A}} $, the *Frobenius norm* of a matrix.

Let our cost function $f(X)$ be defined for $n\times p$ matrices $X$ as follows. Let $C$ be a fixed symmetric $n\times n$ matrix (so $C = C^T$). Let $\mu$ be a scalar that is larger than the $p^{th}$ smallest eigenvalue of $C$. Let $N$ be a diagonal $p\times p$ matrix with distinct positive entries on the diagonal.

The cost function is defined as
\begin{equation}
  f(X) = \frac{1}{2} \trace{X^T C X N} + \mu \frac{1}{4} \Norm{N - X^T X}^2_F
\end{equation}
where $ X \in \RR^{n \times p} $, $ n \ge p $.

## Frobenious Norm

Implement a Python function ```frobenius_norm``` which accepts an arbitrary matrix $ A $ and returns
$ \Norm{A}_F $ using the formula given. (Use ```numpy.trace``` and ```numpy.sqrt```.) We represent matrices and vectors as numpy ndarrays.
1. Given a matrix $ A \in \RR^{n \times p} $, what is the complexity of your implementation of ```frobenius_norm```
using the formula above?
2. Can you come up with a faster implementation, if you were additionally told that $ p \ge n $ ?

Extension: Can you find an even faster implementation than in 1. and 2.? 

### Solution


1. The formula above invovles multiplying the matrices as $A^TA$ and then taking the trace. The complexity of the entire operation comes from multipying the matrices, which is of complexity $O(p^2n)$.

2. If $p\geq n$, then as $ \trace{A^T A} = \trace{A A^T}$ we can multiply the matrices as $AA^T$ to get a complexity of $O(n^2p)$. Which property of traces did we use to achieve this identity?

Extension: Note that $\|A\|_F=\sqrt{\sum\limits_{i=1}^p\sum\limits_{j=1}^n A_{j,i}^2}$, which has a complexity of $O(np)$. To see this, note that
\begin{align*}
  \trace{A^T A} &= \sum_{i=1}^p (A^T A)_{i,i} \\
                &= \sum_{i=1}^p \sum_{j=1}^n \underbrace{(A^T)_{i,j}}_{=A_{j, i}} A_{j,i}\\
                &= \sum_{i=1}^p \sum_{j=1}^n A_{j,i}^2.
\end{align*}

We will use this method to implement the Frobenius Norm.


In [2]:
# Solution
def frobenius_norm(A):
    """Calculate the Frobenius norm of an array or matrix.
    A -- array or matrix
    """
    return np.sqrt((np.asarray(A)**2).sum(axis=None))

M = np.random.rand(5,3)
print(M)
print(frobenius_norm(M))

[[0.85762625 0.53286482 0.90607835]
 [0.78284506 0.91066499 0.0643526 ]
 [0.17403402 0.46824711 0.28689979]
 [0.72233001 0.33099489 0.99187648]
 [0.62327875 0.58373964 0.36672811]]
2.4693022946168357


## Implementing the cost function

Write a Python function, ```cost_function_for_matrix```, which implements the function $f(X)$ defined above.

Hint: What should the arguments to this function be?

In [3]:
# Solution
def cost_function_for_matrix(X, C, N, mu):
    """
    Calculate the cost function at point X given as a matrix.
    X -- current point in matrix form
    C -- symmetric matrix
    N -- diagonal matrix
    mu -- scalar
    returns the value of the cost function as a scalar.
    """
    if not isinstance(X, np.matrix):
        raise TypeError("X is not a matrix")

    if not isinstance(C, np.matrix):
        raise TypeError("C is not a matrix")

    if not isinstance(N, np.matrix):
        raise TypeError("N is not a matrix")

    r1 = 0.5  * np.trace(X.T * C * X * N)
    r2 = 0.25 * mu * frobenius_norm(N - X.T * X)**2
    return r1 + r2

## Cost function with vector argument

The standard optimisation functions we will be using work only for cost functions that take a vector as the varying argument. Write a new function, ```cost_function_for_vector```, that takes $X$ represented as a vector of length $np$ rather than a matrix of dimensions $n\times p$. What arguments will this function take?

In [4]:
# Solution
def cost_function_for_vector(X, C, N, mu, n, p):
    """Calculate the cost function at point X given as 1-D array
    X  -- current point as 1-D array
    C  -- symmetric matrix
    N  -- diagonal matrix
    mu -- scalar
    n  -- row dimension of X
    p  -- column dimension of X
    returns the value of the cost function as a scalar
    """
    if not isinstance(X, np.ndarray):
        raise TypeError("X is not a matrix")

    if X.ndim != 1:
        raise ValueError("X is not a 1-D vector")

    Xmatrix = np.matrix(X.reshape((n, p)))
    return cost_function_for_matrix(Xmatrix, C, N, mu)

## Minimising the cost function

At this point, we have two main choices in how we minimise the cost function using gradient descent functions from ``scipy.optimize``. First, we can use ``fmin``, which takes a function to minimize and an initial value. Second, we can use ``fmin_bfgs``, which takes an additional argument: the gradient of the function. As a result, (we would expect to find that) ``fmin_bfgs`` has substantially faster convergence.

### Minimizing with ```fmin```

Implement a function ```minimise_f_using_fmin``` that, for given values of $C$, $N$ and $\mu$, finds the matrix $X$ that minimizes $f(X)$ using ``fmin``. You will likely need [the docs for ``fmin``](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html). Check if your function converges for some (randomly generated) values of $C$, $N$ and $\mu$.

Summary of the docs: if you have a cost function $g(x, y)$ with a fixed value of $y$ and wish to find the value of $x$ that minimizes it, the syntax for calling ``fmin`` would be ``fmin(g, x0, args=(y))`` where ``x0`` is an initial guess for the value of $x$, and ``args=(y)`` gives ``fmin`` the rest of the values to pass to the cost function. Note that it is necessary that the variable that can change is the first argument to the cost function.

In [5]:
# Solution

def minimise_f_using_fmin(C, N, mu, n, p, X0):
    """Run minimisation with simplex algorithm."""

    X_at_min = opt.fmin(cost_function_for_vector,
                                 X0,
                                 args=(C, N, mu, n, p))
    X_at_min = np.matrix(X_at_min.reshape((n, p)))
    return X_at_min

def randomly_generate_values(n, p):
    C = np.random.rand(n,n)
    C = np.matrix(C+C.T)
    N = np.matrix(np.diag(np.random.rand(p)))
    mu = frobenius_norm(C)
    X0 = np.random.randn(n,p)
    return C, N, mu, X0

np.random.seed(100)

n = 3
p = 2
C, N, mu, X0 = randomly_generate_values(n, p)
minimise_f_using_fmin(C, N, mu, n, p, X0)

Optimization terminated successfully.
         Current function value: -0.463333
         Iterations: 554
         Function evaluations: 857


matrix([[-0.55754315,  0.20111999],
        [ 0.22288385, -0.89391836],
        [ 0.55312147,  0.56292356]])

### Calculating the gradient of the cost function

To use ``fmin_bfgs``, which is substantially more time efficient, we need to compute the gradient of $f(X)$ with respect to $X$. Calculate this gradient, then implement a function to calculate it. You may want to use Sam Roweis' [Matrix Identities](https://cs.nyu.edu/~roweis/notes/matrixid.pdf) and/or the [Matrix Cookbook](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf) as a reference for matrix calculus. As our cost function uses its main argument $X$ represented as a vector, also implement a function ```gradient_for_vector``` which returns the gradient represented as a vector.

### Solution

The cost function was defined as
\begin{equation}
  f(X) = \frac{1}{2} \trace{X^T C X N} + \mu \frac{1}{4} \Norm{N - X^T X}^2_F
\end{equation}
where $ X \in \RR^{n \times p} $, $ n \ge p $.

You should be able to show that 
\begin{align*}
    \nabla_X \trace{X^TCXN} 
        &= CXN + C^TxN^T\\
        &= 2CXN \tag{as $C$ and $N$ are symmetric}
\end{align*}
and that
\begin{align*}
    \nabla_X \Norm{N - X^T X}^2_F 
        &= -2XN^T-2XN + 4 XX^TX\\
        &= -4XN+4XX^TX \tag{as $N$ is symmetric}\\
        &= -4X(N-X^TX).
\end{align*}

This can be done by calculating the partial derivative for an arbitrary $X_{ij}$ and generalising.

For example, note that
\begin{align*}
    \frac{\partial}{\partial X_{ij}} \trace{X^TCXN} 
        &= \frac{\partial}{\partial X_{ij}} \sum_k\sum_l\sum_m\sum_n X_{lk}C_{lm}X_{mn}N_{nk}\\
        &= \sum_m\sum_n C_{im}X_{mn}N_{nj} + \sum_k\sum_l X_{lk}C_{li}N_{jk}\\
        &= (CXN)_{ij} + (C^TXN^T)_{ij}
\end{align*}
so $$\nabla_X \trace{X^TCXN} = CXN + C^TXN^T.$$

To check this, from the [matrix cookbook](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf) we have the identity (page 13, equation 117)
$$\nabla_X\trace{X^TBXC} = BXC+B^TXC^T$$
so it follows that
$$\nabla_X \trace{X^TCXN} = CXN + C^TXN^T.$$

Thus we have that
\begin{align*}
    \nabla_X f(X)
        &= CXN - \mu X(N-X^TX).
\end{align*}

In [6]:
# Solution
def gradient_for_matrix(X, C, N, mu):
    """Calculate the gradient for the cost function at a point X
    X  -- current point in matrix form
    C  -- symmetric matrix
    N  -- diagonal matrix
    mu -- scalar
    returns the gradient of the cost function as matrix
    """
    gradient = C * X * N - mu * X * (N - X.T * X)
    return gradient

def gradient_for_vector(X, C, N, mu, n, p):
    """Calculate the gradient for the cost function at a point X
    X  -- current point as 1-D array
    C  -- symmetric matrix
    N  -- diagonal matrix
    mu -- scalar
    n  -- row dimension of X
    p  -- column dimension of X
    returns the gradient of the cost function as 1-D array
    """
    Xmatrix = np.matrix(X.reshape((n, p)))
    gradient =  gradient_for_matrix(Xmatrix, C, N, mu)
    return np.asarray(gradient).reshape((n*p,))



### Minimizing the cost function using the gradient

Write a function ```minimise_f_using_fmin_bfgs``` to minimise $f(X)$ using ```fmin_bfgs```. Have a look at the docs to find the correct syntax. Again, have a try of your function to check that it converges.

In [7]:
# Solution
def minimise_f_using_fmin_bfgs(C, N, mu, n, p, X0):
    """Run minimisation with BFGS algorithm"""

    X_at_min = opt.fmin_bfgs(cost_function_for_vector,
                                 X0,
                                 fprime=gradient_for_vector,
                                 args=(C, N, mu, n, p))
    X_at_min = np.matrix(X_at_min.reshape((n, p)))
    return X_at_min
    
np.random.seed(100)
n = 3
p = 2
C, N, mu, X0 = randomly_generate_values(n, p)
minimise_f_using_fmin_bfgs(C, N, mu, n, p, X0)

Optimization terminated successfully.
         Current function value: -0.463333
         Iterations: 17
         Function evaluations: 19
         Gradient evaluations: 19


matrix([[-0.55754071, -0.20112249],
        [ 0.22287441,  0.8939319 ],
        [ 0.5531515 , -0.56290044]])

## Time for convergence

We wish to check whether ``fmin_bfgs`` is actually faster than ``fmin``.

First, we need a way of randomly generating input parameters for our cost function.

### Construction of a random matrix $C$ with given eigenvalues

A diagonal matrix has the nice property that the eigenvalues can be directly read off
the diagonal. Given a diagonal matrix $ C \in \RR^{n \times n} $ with distinct eigenvalues, 
how many different diagonal matrices have the same set of eigenvalues?

Given a diagonal matrix $ C \in \RR^{n \times n} $ with distinct eigenvalues,
how many different matrices have the same set of eigenvalues?

Given a set of $ n $ distinct real eigenvalues $ \mathcal{E} = \{e_1, \dots, e_n \} $, 
write a Python function ```random_matrix_from_eigenvalues``` which takes a list of
eigenvalues $ E $ and returns a random symmetric matrix $ C $ having the same eigenvalues.

### Solution

There are $ n! $ permutations of diagonal elements, but infinitely many matrices
with the same set of eigenvalues (can scale with any positive number).

In order to construct a random matrix with given eigenvalues $\lambda_i$, $i=1,\dots,n$
we first create a diagonal matrix $ \Lambda $ with those eigenvalues on the
diagonal. Then we can get another matrix $ A $ with the same eigenvalues as $ \Lambda $
if we apply an arbitrary nonsingular matrix $ B $ to get $ A = B \Lambda B^{-1} $.
(Can you prove that $ A $ and $ \Lambda $ have the same set of eigenvalues?)

If $ B $ is an orthogonal matrix $ Q $, then $ Q^{-1} = Q^T $ and therefore the above
formula results in $ A = Q \Lambda Q^T $ which is much faster to calculate then
using the inverse of a matrix.

How to get a random orthogonal matrix? We use the QR-decomposition of a matrix which 
decomposes every arbitrary matrix $ B $ into an orthogonal matrix $ Q $ and an 
upper-triangular matrix $ R $, $ B = Q R $.

The algorithm therefore is
1. Choose a random matrix $ B $ by randomly choosing the elements of $ B $.
2. Calculate the QR-decomposition $ B = Q R $. (Check that $ B $ is nonsingular
      by checking the diagonal of $ R $ has nonzero elements.)
3. Calculate $ A =  Q \Lambda Q^T $, the wanted arbitrary matrix with the
      same eigenvalues as $ \Lambda $.

In [8]:
# Solution
def random_matrix_from_eigenvalues(E):
    """Create a square random matrix with a given set of eigenvalues
    E -- list of eigenvalues
    return the random matrix
    """
    n    = len(E)
    # Create a random orthogonal matrix Q via QR decomposition
    # of a random matrix A
    A    = np.matrix(np.random.rand(n,n))
    Q, R = np.linalg.qr(A)
    #  similarity transformation with orthogonal
    #  matrix leaves eigenvalues intact
    return Q * np.diag(E) * Q.T


### Checking convergence time

Is ``fmin_bfgs`` actually faster than ``fmin``? Write some code to find out, using ```time.clock()```.

Make sure to check this for relatively small and relatively large values of $n$ and $p$. Use ``random_matrix_from_eigenvalues`` to generate your $C$ parameter.

In [9]:
# Solution
np.random.seed(1)
def initialise_low_dimensional_data():
    """Initialise the data, low dimensions"""
    n = 3
    p = 2
    mu = 2.7

    N = np.matrix(np.diag([2.5, 1.5]))
    E = [1, 2, 3]
    C = random_matrix_from_eigenvalues(E)
    X0 = np.random.rand(n*p)

    return C, N, mu, n, p, X0


def initialise_higher_dimensional_data():
    """Initialise the data, higher dimensions"""
    n  = 20
    p  =  5
    mu = p + 0.5

    N = np.matrix(np.diag(np.arange(p, 0, -1)))
    E = np.arange(1, n+1)
    C = random_matrix_from_eigenvalues(E)
    X0 = np.random.rand(n*p)

    return C, N, mu, n, p, X0

def pretty_printing(task_string):
    line_length  = 76
    spaces       = 2
    left_padding = (line_length - len(task_string)) // 2
    right_padding = line_length - left_padding - len(task_string)
    print("=" * line_length)
    print("=" * (left_padding - spaces) + " " * spaces + task_string + \
            " " * spaces + "=" * (right_padding - spaces))
    print("=" * line_length)    

def run_and_time_all_tests():
    """Run all test and time them using a list of function names"""
    List_of_Test_Names = ["minimise_f_using_fmin",
                 "minimise_f_using_fmin_bfgs"]

    List_of_Initialisations = ["initialise_low_dimensional_data",
                               "initialise_higher_dimensional_data"]

    for test_name in List_of_Test_Names:
        for init_routine in List_of_Initialisations:
            task_string  = test_name + "(" + init_routine + ")"
            pretty_printing(task_string)

            start = time.clock()
            C, N, mu, n, p, X0 = globals()[init_routine]()
            exec(test_name+"(C,N,mu,n,p,X0)")
            run_time = time.clock() - start
            print("run_time :", run_time)

run_and_time_all_tests()

=========  minimise_f_using_fmin(initialise_low_dimensional_data)  =========
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 439
         Function evaluations: 681
run_time : 0.1660790000000003
=======  minimise_f_using_fmin(initialise_higher_dimensional_data)  ========
run_time : 7.041362
======  minimise_f_using_fmin_bfgs(initialise_low_dimensional_data)  =======
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 12
         Function evaluations: 18
         Gradient evaluations: 18
run_time : 0.01038600000000045
=====  minimise_f_using_fmin_bfgs(initialise_higher_dimensional_data)  =====
Optimization terminated successfully.
         Current function value: 40.727273
         Iterations: 128
         Function evaluations: 137
         Gradient evaluations: 137
run_time : 0.2757680000000011


## Minima of $f(X)$

Compare the columns $x_1,\dots, x_p$ of the matrix $X^\star$ which minimises $ f(X) $ 
\begin{equation}
  X^\star = \argmin_{X \in \RR^{n \times p}} f(X)
\end{equation}

with the eigenvectors related to the smallest eigenvalues of $ C $.

What do you believe this means about $f(X)$?


### Solution

From the code below we see that the normalised columns of $X^\star$ 
correspond to eigenvectors of $C$.

In fact, the minimum of the given cost function are matrices $ X $ which contain
the $ p $ eigenvectors of $ C $ which are associated with the $ p $ smallest
eigenvalues of $ C $. The order of the eigenvector in the minimum $ X $
is defined by the order of the diagonal elements in $ N $.

In [10]:
# Solution
np.random.seed(5)
def normalize_columns(A):
    """Normalise the columns of a 2-D array or matrix to length one
    A - array or matrix (which will be modified)
    """
    if A.ndim != 2:
        raise ValueError("A is not a 2-D array")

    number_of_columns = A.shape[1]
    for i in range(number_of_columns):
        A[:,i] /= np.linalg.norm(A[:,i], ord=2)


def show_results(X_at_min, C):
    """Display the found arg min and compare with eigenvalues of C
    X_at_min -- arguement at minimum found
    C        -- symmetric matrix
    """
    n,p = X_at_min.shape

    normalize_columns(X_at_min)

    # Get the eigenvectors belonging to the smallest eigenvalues
    Eigen_Values, Eigen_Vectors = np.linalg.eig(C)
    Permutation = Eigen_Values.argsort()
    Smallest_Eigenvectors = Eigen_Vectors[:, Permutation[:p]]

    if n < 10:
        print("X_at_min               :\n", X_at_min)
        print()
        print("Smallest_Eigenvectors  :\n", Smallest_Eigenvectors)
        print()
    else:
        Project_into_Eigenvectorspace = \
          Smallest_Eigenvectors * Smallest_Eigenvectors.T * X_at_min
        Normal_Component = X_at_min - Project_into_Eigenvectorspace

        print("norm(Normal_Component)/per entry :", \
            np.linalg.norm(Normal_Component, ord=2) / float(n*p))

def show_comparision(num=3):
    for _ in range(num):
        pretty_printing("Comparing X_at_min and Eigenvalues for random values")
        C, N, mu, n, p, X0 = initialise_low_dimensional_data()
        X_at_min = minimise_f_using_fmin_bfgs(C,N,mu,n,p,X0)
        show_results(X_at_min, C)

show_comparision()

==========  Comparing X_at_min and Eigenvalues for random values  ==========
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 19
         Function evaluations: 24
         Gradient evaluations: 24
X_at_min               :
 [[ 0.18249115 -0.9764946 ]
 [ 0.75515978  0.21391152]
 [ 0.62962741  0.02646037]]

Smallest_Eigenvectors  :
 [[-0.18249318 -0.97649328]
 [-0.75515939  0.21391723]
 [-0.6296273   0.0264629 ]]

==========  Comparing X_at_min and Eigenvalues for random values  ==========
Optimization terminated successfully.
         Current function value: 3.962963
         Iterations: 14
         Function evaluations: 19
         Gradient evaluations: 19
X_at_min               :
 [[ 0.37258559 -0.89968829]
 [ 0.85475654  0.42815364]
 [ 0.36134644 -0.08512011]]

Smallest_Eigenvectors  :
 [[-0.37258535 -0.89968729]
 [-0.85475642  0.42815549]
 [-0.36134696 -0.08512138]]

==========  Comparing X_at_min and Eigenvalues for random values  